# Brain Data Get
- Obtain number of GW, WM, & CSF Pixels from segmented images

In [1]:
# Pathing Libraries
from pathlib import Path
import os
import os.path as op
import glob

# Image Libraries
import nibabel as nib
import numpy as np
import pandas as pd
import scipy.ndimage as ndi
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
## Pathing
path = '../'

images_path = Path(path, 'data', 'images')
masks_path = Path(path, 'data', 'masks')
segs_path = Path(path, 'data', 'segs_refs')
metadata_path = Path(path, 'data', 'meta')

## Functions
def z_normalize(img_data, mask_data):
    # Ensure the mask is binary
    mask_data = np.where(mask_data > 0, 1, 0)

    # Apply the mask to the image
    masked_image_data = img_data * mask_data

    # Calculate the z-score of the masked image
    masked_image_data[mask_data==0] = stats.zscore(masked_image_data[mask_data==0])

    # Handle NaN values that might result from zscoring zero-valued elements
    masked_image_data = np.nan_to_num(masked_image_data)

    return masked_image_data


In [29]:
meta_data = pd.read_csv(Path(metadata_path, 'meta_data_all.csv'))


for subj, age, gender in zip(meta_data['subject_id'], meta_data['age'], meta_data['gender_text']): # ''gender_code'; 1= male, 2= female
    print(subj, age, gender)

    # Loading in Segmentation Data
    seg = nib.load(op.join(segs_path, 'sub-' + subj + '_T1w_seg.nii.gz'))
    seg_data = seg.get_fdata()

    # Loading in Mask Data
    mask = nib.load(op.join(masks_path, 'sub-' + subj + '_T1w_brain_mask' + '.nii.gz'))
    mask_data = mask.get_fdata()

    # Z-score Normalization
    z_seg = z_normalize(seg_data, mask_data)

    # Count CSF (1), grey matter (2), and white matter (3) pixels
    num_ones = np.count_nonzero(z_seg == 1)
    num_twos = np.count_nonzero(z_seg == 2)
    num_threes = np.count_nonzero(z_seg == 3)

    print(f'{subj} CSF: {num_ones} GM: {num_twos} WM: {num_threes}')
    
    # Combining participant data for ML classification





CC110033 24 MALE
CC110033 CSF: 32255 GM: 88877 WM: 53097
CC110037 18 MALE
CC110037 CSF: 23522 GM: 93552 WM: 53059
CC110045 24 FEMALE
CC110045 CSF: 22655 GM: 97481 WM: 49497
CC110056 22 FEMALE
CC110056 CSF: 20673 GM: 86147 WM: 49316
CC110062 20 MALE
CC110062 CSF: 20466 GM: 110771 WM: 60077
CC110069 28 FEMALE
CC110069 CSF: 33716 GM: 87070 WM: 50853
CC110087 28 FEMALE
CC110087 CSF: 25119 GM: 87639 WM: 45359
CC110098 23 MALE
CC110098 CSF: 31622 GM: 101799 WM: 53795
CC110101 23 MALE
CC110101 CSF: 32199 GM: 105892 WM: 55422
CC110126 22 FEMALE
CC110126 CSF: 22748 GM: 100499 WM: 55779
CC110174 25 FEMALE
CC110174 CSF: 16458 GM: 75982 WM: 43297
CC110182 18 FEMALE
CC110182 CSF: 25082 GM: 92165 WM: 45913
CC110187 25 FEMALE
CC110187 CSF: 25661 GM: 90829 WM: 48961
CC110319 28 FEMALE
CC110319 CSF: 23648 GM: 105225 WM: 60920
CC110411 25 MALE
CC110411 CSF: 39304 GM: 107591 WM: 62531
CC110606 20 MALE
CC110606 CSF: 36672 GM: 107434 WM: 59233
CC112141 29 MALE
CC112141 CSF: 35605 GM: 95060 WM: 59303
CC1200

## Machine Learning Loop Reference Code

In [32]:
# Scikit-learn
#from sklearn.utils.fixes import loguniform
from sklearn.utils import compute_class_weight
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import PrecisionRecallDisplay, ConfusionMatrixDisplay, classification_report, make_scorer, balanced_accuracy_score, fbeta_score, precision_recall_curve, precision_score, recall_score, accuracy_score, roc_auc_score, f1_score, matthews_corrcoef, confusion_matrix
# Classifiers
from sklearn import svm
from sklearn.svm import LinearSVC    
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier

In [ ]:
%%time
%xmode Verbose

scaler = StandardScaler()
vectorizer = Vectorizer()

# Making the crossvalidation to be used in the RandomizedSearch
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

for subject in subjects:
    print('\n-------\n\033[;40m' + subject + '\033[m')
    
    # Clearing out the saved data for each participant
    data_table = pd.DataFrame()
    data_table_list = []
    
    for contr, conds in contrasts.items():
        print('-------\n\033[94;40m' + contr + '\033[m')
        subj_epochs = epochs[subject][conds]
        
        # Create a list of labels from event codes mapped to event_id
        event_id_rev = dict(zip(subj_epochs.event_id.values(), subj_epochs.event_id.keys()))
        labels_all = [event_id_rev[e] for e in subj_epochs.events[:, 2]]
        labels_all = pd.DataFrame(labels_all)[0].str.split('/', expand=True).rename(columns={0:'Colour', 1:'Orientation', 2:'Type', 3:'Status', 4:'Location'} )
        label_map = {'Target':1, 'Standard':0}
        labels_all['labels'] = labels_all['Status'].map(label_map)
        labels = labels_all['labels']
        
        # Extract data from subj_epochs and vectorize 
        D = subj_epochs.get_data()
        
        # Create train-test split
        X_train, X_test, y_train, y_test = train_test_split(D, 
                                                            labels,
                                                            stratify=labels,
                                                            test_size=cl_p['test_size'], 
                                                            random_state=42,
                                                            shuffle=True
                                                           )

        # Classifier Loop
        for c_name, c in classifiers.items():
            print('-------\nRunning classifier: \033[1;91;40m' + c_name + '\033[m')

            # Making the Pipeline
            clf = Pipeline([('Vectorizer', vectorizer),
                             ('Scaler', scaler),
                             (c_name, c)                                 
                             ])

            # Cross validating
            cv_cv = cross_validate(clf, X_train, y_train, 
                                   cv=cv,
                                   scoring=scoring,
                                   return_train_score=True, # Determines if Training scores are included in .cv_results_
                                   n_jobs=5,
                                   error_score='raise' # For debugging purposes
                                   )

            print('Training Classifier')
            clf = clf.fit(X_train, y_train)

            print('Predicting...')
            y_pred = clf.predict(X_test)

            print('Scoring...')        
            print(classification_report(y_test, y_pred))

            # Confusion Matrix Generation and Visualization within the loop -> saved to csv as "[[TN, FN] [FP, TP]]"
            cm = confusion_matrix(y_test, y_pred, labels=clf.classes_)
            cmd = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
            cmd.plot()
            plt.title(subject + '_' + str(contr).replace('/', '_') + '_' + c_name)
            plt.show()

            # Saving CV results to a DataFrame 
            results = pd.DataFrame(cv_cv)

            data_table_list.append(pd.DataFrame({'participant_id': subject,
                                              'Condition': contr,
                                              'Classifier': c_name,

                                              # Confusion_matrix saved in format: [[TN, FN] [FP, TP]]
                                              'Confusion_Matrix': str(cm),                                                 

                                              'CV_Train_Bal_Accuracy': results['train_Bal_Acc'].round(3) * 100,
                                              'CV_Test_Bal_Accuracy': results['test_Bal_Acc'].round(3) * 100,
                                              'Test_Bal_Accuracy': round(balanced_accuracy_score(y_test, y_pred), 3) * 100,

                                              'CV_Train_Accuracy': results['train_Acc'].round(3) * 100,
                                              'CV_Test_Accuracy': results['test_Acc'].round(3) * 100,
                                              'Test_Accuracy': round(accuracy_score(y_test, y_pred), 3) * 100, 

                                              'CV_Train_Precision': results['train_Prec'].round(3) * 100,
                                              'CV_Test_Precision': results['test_Prec'].round(3) * 100,                                                
                                              'Test_Precision': round(precision_score(y_test, y_pred, zero_division=0), 3) * 100,    

                                              'CV_Train_Matthews_coef': results['train_Matthews_Coef'].round(3),
                                              'CV_Test_Matthews_coef': results['test_Matthews_Coef'].round(3),
                                              'Matthews_Coef': round(matthews_corrcoef(y_test, y_pred), 3),

                                              'CV_Train_Recall': results['train_Recall'].round(3) * 100,
                                              'CV_Test_Recall': results['test_Recall'].round(3) * 100,
                                              'Test_recall': round(recall_score(y_test, y_pred), 3) * 100,

                                              'CV_Train_Fbeta_0.5': results['train_Fbeta_0.5'].round(3),
                                              'CV_Train_Fbeta_0.5': results['train_Fbeta_0.5'].round(3),
                                              'Fbeta_0.5': round(fbeta_score(y_test, y_pred, beta = 0.5, zero_division=0), 3),

                                              'CV_Train_Fbeta_1.5': results['train_Fbeta_1.5'].round(3),
                                              'CV_Test_Fbeta_1.5': results['test_Fbeta_1.5'].round(3),
                                              'Fbeta_1.5': round(fbeta_score(y_test, y_pred, beta = 1.5, zero_division=0), 3),

                                              'CV_Train_F1': results['train_F1_score'].round(3),
                                              'CV_Test_F1': results['test_F1_score'].round(3),
                                              'F1_score': round(f1_score(y_test, y_pred, zero_division=0), 3),

                                              'CV_Train_ROC_AUC': results['train_ROC'].round(3),
                                              'CV_Test_ROC_AUC': results['test_ROC'].round(3),                                             
                                              'Test_ROC_AUC': round(roc_auc_score(y_test, y_pred), 3),

                                              'Mean Fit Time': results['fit_time'].round(3),
                                              'Mean Score Time': results['score_time'].round(3)
                                             }, index=[0]
                                            )
                               )

    # Saving Data to CSV Per Participant
    data_table = pd.concat(data_table_list)
    data_table.to_csv(f'{str(subject)} new_Data.csv')